In [1]:
import sys
import os
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.config", "4g").appName('entity-relation').getOrCreate()

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
# Q1 Develop a PySpark script to clean and preprocess data before performing entity resolution.
# Include steps like tokenization and normalization.

# Sol) the given data is already normailzed , tokenized (no more possibility of tokenization!)

In [3]:
prev =  spark.read.option("header", "true").option("nullValue", "?").\
option("inferSchema", "true").csv("Data/csv-data/block_1.csv")
prev.show()

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|        null|         1.0|        null|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|        null|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
|42413|48491|              1.0|        null|         1.0|       

In [4]:
prev = prev.fillna(0)
prev.show()

+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|     cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+-----------------+------------+------------+------------+-------+------+------+------+-------+--------+
|37291|53113|0.833333333333333|         0.0|         1.0|         0.0|      1|     1|     1|     1|      0|    true|
|39086|47614|              1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|70031|70237|              1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|84795|97439|              1.0|         0.0|         1.0|         0.0|      1|     1|     1|     1|      1|    true|
|36950|42116|              1.0|         0.0|         1.0|         1.0|      1|     1|     1|     1|      1|    true|
|42413|48491|              1.0|         0.0|         1.0|       

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col


In [ ]:
feature_cols = ["cmp_fname_c1", "cmp_fname_c2", "cmp_lname_c1", "cmp_lname_c2",
                "cmp_sex", "cmp_bd", "cmp_bm", "cmp_by", "cmp_plz"]

# Assemble feature vectors
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(prev)

# Create a Bucketed Random Projection LSH model
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", seed=42, numHashTables=5, bucketLength=1.0)
model = brp.fit(df)

# Transform the data to get hash values
hashed_data = model.transform(df)

# Self-join to find cosine similarity
similarity_df = model.approxSimilarityJoin(hashed_data, hashed_data, threshold=0.6, distCol="cosineSimilarity")

# Filter for positive matches
matches = similarity_df.filter(col("datasetA.id_1") < col("datasetB.id_1"))

# Show the result
matches.select("datasetA.id_1", "datasetA.id_2", "datasetB.id_1", "datasetB.id_2", "cosineSimilarity").show(truncate=False)


In [13]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f3d6e9e1ef0>>